**Libs**:  
Scrapy | cartodb | numpy/pandas/matplotlib/scikit-learn | scikit-image | nltk | (plotly/graphene/etc)  

**Tools**:  
Python | firefox+firebug+firepath | Linux+CLI  

**Pages**:  
Idealista | Segundamano | ?  

**Time**:  
1h50m + 10m (Q/A)  

**Demos**:  
Scrapy | XPATH | panda | matplotlib | S.GIS (Filtering/etc) | CLI |  

# Siri, ¡Búscame un piso!
### Cómo encontrar el piso perfecto en <2h

Yuri Numerov (Achifaifa)  
[achi.se](http://achi.se)  -  [achi (at) hush (.) ai]()

Euskal Encounter 24, 2015-07-24

### Sobre mí

¯\\\_(ツ)\_/¯

### Antes, un poco de spam...

![](./img/logo-gamaker.jpg)

### Y otro poco más:

![](./img/pyladies-logo.png) ![](./img/pyss-logo.png)

### Analicemos el problema.

¿Cómo puede buscar alguien un piso?

#### A) Manera tradicional

* Abrir X pestañas en el navegador con diferentes páginas web

* Buscar por ciudad/CP

* Hacer scroll hasta que

* * Encontremos un piso

* * Nos aburramos 

* * Suframos de Síndrome de Túnel Carpiano

#### B) Manera **guay** 

* Escribimos un programa que lo haga por nosotros

* ???

* Profit

#### ¿A ó B?

![](./img/matrix-pills.jpg)

### ¿Qué herramientas vamos a usar?

#### Herramientas generales:

* [Python](http://python.org)

* Linux (CLI)

* iceweasel (Firefox) + firebug + firepath

* [CartoDB](https://cartodb.com): Servicio de visualización de datos cartográficos

#### Librerías de python:

* [Scrapy](http://scrapy.org): Framework para web scraping

* Geocoder: Librería para geocoding y reverse geocoding

* [Matplotlib](http://matplotlib.org): Librería para generación de gráficos

* [Numpy](http://www.numpy.org): Librería con *muchas* funciones matemáticas extra

* [Pandas](http://pandas.pydata.org): Librería para análisis y manipulación de datos

### ¿Cuál es el plan?

* Localizar webs con suficientes datos disponibles

* Recoger todos los datos de esa web

* Completar los datos

* Filtrar los datos, ordenarlos y visualizarlos

¡Vamos a ello!

¡Venga vámonos!

### 01) Localizando páginas web

Hay webs con la mínima información posible (e.g. segundamano.es)

Necesitamos localizar webs que tengan toda la información que necesitamos

(O información suficiente para inferir el resto)

Dirección/GPS, mail/teléfono, fotos, precio, características...

Lo difícil no es encontrar piso, es encontrar páginas web decentes :_(

Haremos un primer ejemplo con [idealista](http://idealista.com). El resto de páginas se procesan igual ;)

### 02) Scrapy y XPATH

Para cada web tendremos que escribir un programa que recoja la información

Utilizamos Scrapy para no tener que hacerlo todo nosotros

Los datos en las páginas de los pisos son diferentes, pero la estructura de la página es la misma ;)

Podemos automatizar la recogida de webs y acceder a elementos individuales usando [XPath](https://en.m.wikipedia.org/wiki/Xpath)

#### Scrapy en un ratín:

* Creamos una nueva araña con **`scrapy newspider <url>`**

* Especificamos parámetros como las URLs de inicio y dominios permitidos

* Escribimos la/las funciones de parseo

* Ejecutamos la araña con **`scrapy crawl -o <nombre_de archivo>.<extensión> <nombre_de_araña>`**

#### *XPath for dummies*

1) usamos **`//`** para iniciar la expresión  
2) utilizamos diferentes etiquetas para ir seleccionando bloques del HTML de la web  
3) **`*`** es un comodín, **`[@<atributo>=<valor>]`** nos sirve para buscar por atributos  
3b) Podemos conseguir enlaces con **`@href`**, texto con **`text()`**, etc

¡Vamos con algunos ejemplos!

**`//head/title/text()`** Va al principio de la página (**`//`**), ahí busca la cabecera (**`head/`**), en la cabecera busca el título (**`title/`**) y de ese elemento coge el texto (**`text()`**)

**`//*[@id='bottompanel']/div[2]/strong/a[@class='last']/@href`** : 

Busca por toda la página y selecciona cualquier div que tenga una ID llamada 'bottompanel' (**`*[@id='bottompanel']`**), ahí va al segundo div indentado (**`div[2]`**), busca lo que esté con el atributo strong (**`strong/`**) y ahí busca una etiqueta a con clase 'last' (**`a[@class='last']/`**) y devuelve la url de esa etiqueta (**`@href`**)

(Devuelve el link de la última página del hilo de un foro)
![](./img/xpath-last.png)

**Parece** complicado, hasta que escribes dos o tres

En **Firefox** podemos usar los plugins firebug y firepath, que nos permiten buscar y obtener las expresiones xpath para diferentes elementos

![](./img/firepath.png)

![](./img/firepath2.png)

### 03) Escribiendo una araña

Muchas de las arañas de búsqueda siguen un patrón concreto

* Consigue la lista de todos los elementos de esa página

* Procesa todos los elementos uno por uno

* Una vez acabado, repite si hay otra página

Vamos a escribir una araña que procese pisos de [idealista.com](http://idealista.com)

#### Inicializamos la araña con información

In [4]:
import scrapy

In [5]:
class MySpider(scrapy.Spider):
    name='idealista'
    allowed_domains=["idealista.com"]
    start_urls=['https://www.idealista.com/alquiler-viviendas/bilbao-vizcaya/']

![](./img/idealista-pp.png)

#### Procesamos las páginas con la lista de pisos

In [6]:
def parse(self,response):

    flats=[i for i in response.xpath("//a/@href").extract() if "/inmueble/" in i]

    for flat in flats:
        flaturl=response.urljoin(cube)
        yield scrapy.Request(flaturl,callback=self.parseflat)

    nexturl=response.xpath('//*[@class="next"]/a/@href').extract()
    if nexturl: 
        nexturl=response.urljoin(nexturl[0])
        yield scrapy.Request(nexturl, callback=self.parse)

![](./img/idealista-next.png)

#### Procesamos las páginas individuales de cada piso

(Devolviendo los datos que nos interesan)

In [7]:
def parseflat(self,response):

    size=""
    orientation=""
    rooms=""
    misc_data_flat=response.xpath("//*[@id='details']/div[4]/ul/*/text()").extract()
    for i in misc_data_flat:
        if u"m²" in i: size=i
        if u"Orientación" in i: orientation=i
        if "habitaciones" in i: rooms=i

    floor=""
    int_ext=""
    lift=""
    misc_data_bldg=response.xpath("//*[@id='details']/div[5]/ul/*/text()").extract()
    for i in misc_data_bldg:
        if "Planta" in i: floor=i
        if "interior" in i: int_ext="interior"
        if "exterior" in i: int_ext="exterior"
        if "ascensor" in i: lift=i

    addr_approx=response.xpath("//*[@id='main-info']/h1/span/text()")[0].extract().split(' en ')[-1]
    geodata=geocoder.google(addr_approx)
    gps=geodata.latlng
    cp=geodata.postal
    publicdistance="?"
    
    price=response.xpath("//*[@id='details']/div[2]/ul/li[1]/text()")[0].extract()
    contact=response.xpath("//*[@class='contact-phones']/*/p/text()").extract()
    pics=response.xpath("//img/@src").extract()
    deposit=response.xpath("//*[@id='details']/div[2]/ul/li[1]/text()")[0].extract()
    description=response.xpath("//*[@class='commentsContainer']/div/text()")[0].extract()

    return {'size':size,
            'orientation': orientation,
            'rooms': rooms,
            'floor': floor,
            'int_ext': int_ext,
            'lift': lift,
            'addr_approx': addr_approx,
            'gps': gps,
            'cp': cp,
            'publicdistance': publicdistance,
            'price': price,
            'contact': contact,
            'pics': pics,
            'deposit': deposit,
            'description': description}

![](./img/bad-data.png)

¡Y ya está!

![](./img/burnsff.png)

### 04) Aclarar y repetir para el resto de webs

* Por cada web que queramos procesar necesitaremos una araña diferente

* Todas las arañas pueden ejecutarse a la vez y volcar los datos a un mismo fichero

### 05) Introducción a datos en CSV

**`CSV`** quiere decir **`comma separated values`** (Valores separados por comas)

Un fichero CSV consiste en
* Una cabecera con los campos
* Un cuerpo con los datos

Los datos pueden estar separados por comas, espacios, tabulaciones...

Ejemplo: 

```
nick,puesto,lenguaje
achifaifa,BG28,python
```

#### ¿Y los datos desconocidos?

Un campo vacío se expresa no escribiendo nada ahí

Ejemplo:
```
dato conocido 1,dato desconocido,dato conocido 2
123,,abc
```

Este formato es **muy** fácil de procesar con un programa o script

### 05.5) Y JSON

Parecido, sólo que se usan corchetes. Usando el mismo ejemplo:

    [{"nick": "achifaifa", "puesto":"BG28", "lenguaje":"python"}

Para meter varios datos, se usan elementos de una lista en lugar de líneas en el archivo:

    [{"nick": "achifaifa", "puesto":"BG28", "lenguaje":"python"},
    {"nick": "xXx_Sephirot666_xXx", "puesto":"QB07", "lenguaje":"whitespace"}]

### 06) Introducción a GIS (Filtrando por CP)

Para filtrar por barrio podemos comparar las coordenadas GPS con la geometría del barrio.

Las geometrías están disponibles en los datasets públicos de OSM, por ejemplo

#### No, ahora en serio

Hay una librería de python para eso

In [22]:
import geocoder

In [27]:
geocoder.osm("Calle cubo, Vitoria").postal

u'26001'

Podemos comprobar datos de calles y direcciones individuales, incluyendo el CP

### 07) Primera visualización (Introducción a cartodb)

Uno de los atractivos de recoger datos georreferenciados es poder visualizarlos en un mapa

[CartoDB](http://cartodb.com) nos permite hacer esto fácilmente a partir de datasets con campos GPS

#### Ejemplo (idealista.com)

A partir de este dataset... 

![](./img/img-dataset-pisos.png)

Podemos obtener este mapa

![](./img/toiletmap.png)

#### ¿Cómo se usa?

1) Creamos un dataset (Por ejemplo, con una araña ;P)

2) Lo subimos a la web

3) Creamos un mapa y especificamos el tipo de visualización, mapa, etc

¡Y ya está!

[https://achifaifa.carto.com/viz/17106e8c-4dfd-11e6-ade8-0ee66e2c9693/public_map](https://achifaifa.carto.com/viz/17106e8c-4dfd-11e6-ade8-0ee66e2c9693/public_map)

![](./img/cartodb-rent.png)

### 08) Datos sucios (Completando GPS)

Muchas de las páginas no tienen datos GPS (O tienen datos GPS y no otros, o tienen datos erróneos...)

Necesitamos completar los datos que faltan

* `geocoding` (GPS sobre sobre dirección)

* `reverse geocoding` (Calles, CP, ciudades, países, etc sobre coordenadas GPS) 

#### Geocoding en python

Hay **muchas** maneras de hacerlo. 

Desde descargarnos todo el dataset de OSM (>600GB) hasta usar una API o una librería

La manera más fácil es usar (otra vez) la librería geocoder

In [1]:
import geocoder

In [21]:
geocoder.osm("BEC, Barakaldo").latlng

[43.285405, -2.9906238]

In [19]:
print geocoder.reverse((43.285405, -2.9906238), provider='osm').address

Parking B, Ismael Gorostiza hiribidea, La Paz, Retuerto, Barakaldo, Bizkaia, Euskadi, 48902, España


Y ya está! Podemos añadir esta funcionalidad a nuestra araña para evitar tener que procesar los datos después

### 09) Datos conseguidos. ¿Qué buscamos? (Piscina, jardín, precio...)

Al explorar/procesar datos, es muy importante tener las cosas claras:

#### ¿Qué pregunta queremos responder? ¿Qué duda queremos resolver? ¿Cuál es nuestro objetivo?

En nuestro caso, queremos saber cuál es el mejor piso.

#### ¿Qué es un buen piso?

Las páginas immobiliarias no tienen un campo `piso_bueno` que podamos comprobar.

Necesitamos aclarar qué es para nosotros un piso bueno.

* Tamaño (En este caso, entre 70 y 100m²)

* Altura (Daremos más prioridad a immuebles en pisos muy altos)

* Distancia a transporte público (En este caso, distancia a la parada de metro más cercana)

* Estado del piso (Daremos más peso a pisos nuevos que a pisos de 30 años)

* N° de habitaciones y baños (1..3, 1..2)

* Gastos de comunidad, cuarto de bicis, orientación, etc etc

### 10) Asignando pesos a variables

Una vez sepamos exactamente lo que queremos, podemos asignar pesos a las variables.

El peso de una variable define su importancia dentro del conjunto de variables.

e.g.:  
Tamaño -> peso 10, Habitaciones -> peso 1  
Con este criterio, un piso más grande será preferible a un piso con muchísimas más habitaciones  

Tendremos que elegir cómo de importante son las diferentes características para nosotros (Y ajustarlo según veamos los resultados)

* Tamaño: 8

* Precio: 10

* Altura: 6

* Distancia T.P.: 7

* Estado: 6

* Habitaciones: 3

* Comunidad: 0 (Pagamos alquiler, no vamos a comprar ;P)

* Cuarto de bicis: 4

* Orientación: 5

### 11) Primera tentativa a lo burro

Normalmente se usan técnicas parecidas en el ámbito del aprendizaje máquina (`machine learning`)

Como es nuestro primerito día y estamos en Bilbao, vamos a hacerlo a lo bruto

Colocaremos los "puntos" que da cada parámetro, y los sumaremos con los pesos

In [1]:
valores={
    "tamano": {"71-100": 10, "60-70": 6, "101-120": 6},
    "precio": {"0-500": 10, "501-700": 9, "701-900": 7, ">901": 4},
    "altura": {">5": 10, "3-4": 8, "2": 4, "1": 2},
    "habitaciones": {"1": 2, "2": 10, "3": 8, ">3": 4}
}

In [2]:
pesos={
    "tamano": 8,
    "precio": 10,
    "altura": 6,
    "habitaciones": 3
}

In [3]:
import json
with open("./data/pisos.json", "r") as infile:
    pisos=json.load(infile)

In [13]:
def tamano_piso(piso):
    raw=piso["size"].split(", ")
    try: 
        return int(raw[-1].split()[0].strip())
    except: 
        return -1

In [14]:
def precio_piso(piso):
    raw=piso["price"]
    try:
        return int(raw.split()[0].strip())
    except:
        return -1

In [15]:
def altura_piso(piso):
    raw=piso["floor"]
    try:
        return int(raw.split()[1][:-1])
    except:
        return -1

In [23]:
def habitaciones_piso(piso):
    raw=piso["rooms"]
    try:
        return int(raw.split()[0].strip())
    except:
        return -1

In [17]:
def eval_piso(piso):
    t=tamano_piso(piso)
    p=precio_piso(piso)
    a=altura_piso(piso)
    h=habitaciones_piso(piso)
    
    tamstr="71-100"  if t>70 and t<110 else "60-70" if t<71 and t>59 else "101-120" if t>100 and t<121 else "?"
    prestr="0-500" if p>0 and p<501 else "501-700" if p>500 and p<701 else "701-900" if p>700 and p<901 else ">901"
    altstr=">5" if a>5 else "3-4" if a in [3,4] else str(a) if a>0 else ""
    habstr=">3" if h>3 else str(h) if h>0 else "?"
    
    return {"tamano":tamstr,
            "precio":prestr,
            "altura": altstr,
            "habitaciones": habstr}
    

In [56]:
pisos_rated=[]
for oferta in pisos:
    peso=0
    datos_oferta=eval_piso(oferta)
    for c in ["tamano", "precio", "altura", "habitaciones"]:
        peso+=(valores[c].get(datos_oferta[c], 5)*pesos[c])
    pisos_rated.append((peso, oferta))

pisos_sorted=sorted(pisos_rated, reverse=True)

In [61]:
for item in pisos_sorted[:5]:
    d=item[1]

    print """Score: %i, Location: %s, Price: %s

    %s

    %s %s %s
    %s %s 
    Location: %s (%s)
    
    Contact: %s
    
    =============================================="""%(item[0], d['addr_approx'], d['price'], 
              d['description'], 
              d['floor'], d['lift'], d['int_ext'],
              d['orientation'], d['size'], 
              d['addr_approx'], " ".join([str(i) for i in d['gps']]),
              d['contact'][0]
             )

Score: 260, Location: calle santa clara, 5, Santutxu-Basarrate, Bilbao, Price:  675 eur/mes - 

     "Piso de 80m2. de 2 dormitorios, gran salón independiente con acceso a terraza de 10m2, cocina amueblada y equipada, 1 baño, calefacción individual de gas natural y armarios empotrados. Piso exterior y amueblado. Gastos de comunidad incluidos en el precio de la renta. Finca con ascensor. Piso a 2 minutos del Metro Santutxu, cercano a toda la zona comercial y de ocio." 

     Planta 7ª exterior   Con ascensor  exterior
      80 m² construidos  
    Location: calle santa clara, 5, Santutxu-Basarrate, Bilbao (43.2630126 -2.9349852)
    
    Contact:  902 375 777 
    
Score: 254, Location: julián gayarre, Otxarkoaga - Txurdinaga, Bilbao, Price:  700 eur/mes - 

     "Bonito piso, exterior, luminoso y muy amplio. Consta de 3 habitaciones, cocina equipada, salón grande 1 baño y 1 aseo. Con calefacción individual a gas natural, ventanas con acristalamiento de climalit, suelos de parquet y par

In [65]:
for item in pisos_sorted[:-5:-1]:
    d=item[1]

    print """Score: %i, Location: %s, Price: %s

    %s

    %s %s %s
    %s %s 
    Location: %s (%s)
    
    Contact: %s
    
    =============================================="""%(item[0], d['addr_approx'], d['price'], 
              d['description'], 
              d['floor'], d['lift'], d['int_ext'],
              d['orientation'], d['size'], 
              d['addr_approx'], " ".join([str(i) for i in d['gps']]),
              d['contact'][0]
             )

Score: 104, Location: Campuzano, Bilbao, Price:  1.600 eur/mes - 

     "PISO DE 135 M2 EN INDAUTXU, SALÓN-COMEDOR DE 30 M2, 4 HABITACIONES, 2 BAÑOS, COCINA EQUIPADA, CALEFACCIÓN GAS NATURAL INDIVIDUAL, ASCENSOR, HORMIGÓN." 

     Planta 1ª exterior   Con ascensor  exterior
      145 m² construidos, 135 m² útiles  
    Location: Campuzano, Bilbao (43.3418599 -4.034507)
    
    Contact:  944 941 086 
    
Score: 104, Location: Ensanche-Moyua, Bilbao, Price:  1.500 eur/mes - 

     "Piso en alquiler, para oficinas, despachos..., Dispone de una superficie de 170 m², salón, 4 habitaciones, cocina, 2 baños, calefacción y a/c , con ascensor, exterior." 

     Planta 1ª exterior   Con ascensor  exterior
      170 m² construidos  
    Location: Ensanche-Moyua, Bilbao (43.2630126 -2.9349852)
    
    Contact:  944 947 561 
    
Score: 104, Location: San Pedro de Deusto-Arangoit, Bilbao, Price:  1.300 eur/mes - 

     "Vivienda destinada al alquiler de estudiantes en Sarriko. 4 dormitorios, sal

### 12) Resultados, mejora, etc

Las arañas de búsqueda que hagamos no harán todo bien a la primera

¿Los campos erróneos son por nuestra araña o por la falta de estructura de la web?

Necesitaremos hacer [debug](PENDING URL) a nuestras arañas sin procesar todos los datos otra vez

Es **muy** fácil que pequeños errores pasen desapercibidos. ¡Estad al loro!

GIS es un mundo embarrado y pantanoso, pero es una herramienta útil

### Conclusiones

Consejos, fuentes, información extra, etc

# ¿Dudas? ¿Preguntas?

# Siri, ¡Búscame un piso!
### ¡Gracias! <3

Yuri Numerov (Achifaifa)  
[achi.se](http://achi.se)  -  [achi (at) hush (.) ai]()

Euskal Encounter 24, 2015-07-24